# About this kernel 

+ eca_nfnet_l0
+ ArcFace
+ Mish() activation
+ Ranger (RAdam + Lookahead) optimizer
+ margin = 0.9

## Imports

In [1]:
import sys

sys.path.append('../input/shopee-competition-utils')
sys.path.insert(0,'../input/pytorch-image-models')

In [2]:
import numpy as np 
import pandas as pd 

import torch 
from torch import nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from custom_scheduler import ShopeeScheduler
from custom_activation import replace_activations, Mish
from custom_optimizer import Ranger
from loss_module import ArcMarginProduct

import math 
import cv2
import timm 
import os
import random
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm 

## Config

In [3]:
class CFG: 
    
    DATA_DIR = '../input/shopee-product-matching/train_images'
    TRAIN_CSV = '../input/shopee-product-matching/train.csv'

    # data augmentation
    IMG_SIZE = 512
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    SEED = 2021

    # data split
    N_SPLITS = 5
    TEST_FOLD = 0
    VALID_FOLD = 1

    EPOCHS = 8
    BATCH_SIZE = 8

    NUM_WORKERS = 4
    DEVICE = 'cuda:3'

    CLASSES = 6609 
    SCALE = 30
    MARGIN = 0.9

    MODEL_NAME = 'eca_nfnet_l0'
    MODEL_PATH = f'{MODEL_NAME}_arc_face_epoch_{EPOCHS}_bs_{BATCH_SIZE}_margin_{MARGIN}.pt'
    FC_DIM = 512
    SCHEDULER_PARAMS = {
            "lr_start": 1e-5,
            "lr_max": 1e-5 * 32,
            "lr_min": 1e-6,
            "lr_ramp_ep": 5,
            "lr_sus_ep": 0,
            "lr_decay": 0.8,
        }

## Augmentations

In [4]:
def get_train_transforms():
    return albumentations.Compose(
        [   
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=120, p=0.8),
            albumentations.RandomBrightness(limit=(0.09, 0.6), p=0.5),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():

    return albumentations.Compose(
        [
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0)
        ]
    )

def get_test_transforms():

    return albumentations.Compose(
        [
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0)
        ]
    )

## Reproducibility

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True # set True to be faster

seed_everything(CFG.SEED)

## Dataset 

In [6]:
class ShopeeDataset(torch.utils.data.Dataset):
    """for training
    """
    def __init__(self,df, transform = None):
        self.df = df 
        self.root_dir = CFG.DATA_DIR
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):

        row = self.df.iloc[idx]

        img_path = os.path.join(self.root_dir,row.image)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row.label_group

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return {
            'image' : image,
            'label' : torch.tensor(label).long()
        }

In [7]:
class ShopeeImageDataset(torch.utils.data.Dataset):
    """for validating and test
    """
    def __init__(self,df, transform = None):
        self.df = df 
        self.root_dir = CFG.DATA_DIR
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):

        row = self.df.iloc[idx]

        img_path = os.path.join(self.root_dir,row.image)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row.label_group

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']     
                
        return image,torch.tensor(1)

In [8]:
class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.CLASSES,
        model_name = CFG.MODEL_NAME,
        fc_dim = CFG.FC_DIM,
        margin = CFG.MARGIN,
        scale = CFG.SCALE,
        use_fc = True,
        pretrained = True):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'resnet' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'resnext' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'nfnet' in model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        if use_fc:
            self.dropout = nn.Dropout(p=0.0)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.final = ArcMarginProduct(final_in_features, 
                                           n_classes, 
                                           s=scale, 
                                           m=margin)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        logits = self.final(feature,label)
        return logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x


## ArcMarginProduct

In [9]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CFG.DEVICE)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output, nn.CrossEntropyLoss()(output,label)

## Engine

In [10]:
def train_fn(model, data_loader, optimizer, scheduler, i):
    model.train()
    fin_loss = 0.0
    tk = tqdm(data_loader, desc = "Epoch" + " [TRAIN] " + str(i+1))

    for t,data in enumerate(tk):
        for k,v in data.items():
            data[k] = v.to(CFG.DEVICE)
        optimizer.zero_grad()
        _, loss = model(**data)
        loss.backward()
        optimizer.step() 
        fin_loss += loss.item() 

        tk.set_postfix({'loss' : '%.6f' %float(fin_loss/(t+1)), 'LR' : optimizer.param_groups[0]['lr']})

    scheduler.step()

    return fin_loss / len(data_loader)

def eval_fn(model, data_loader, i):
    model.eval()
    fin_loss = 0.0
    tk = tqdm(data_loader, desc = "Epoch" + " [VALID] " + str(i+1))

    with torch.no_grad():
        for t,data in enumerate(tk):
            for k,v in data.items():
                data[k] = v.to(CFG.DEVICE)
            _, loss = model(**data)
            fin_loss += loss.item() 

            tk.set_postfix({'loss' : '%.6f' %float(fin_loss/(t+1))})
        return fin_loss / len(data_loader)

In [11]:
def read_dataset():
    df = pd.read_csv(CFG.TRAIN_CSV)
    df['matches'] = df.label_group.map(df.groupby('label_group').posting_id.agg('unique').to_dict())
    df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

    gkf = GroupKFold(n_splits=CFG.N_SPLITS)
    df['fold'] = -1
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=df, groups=df['label_group'])):
        df.loc[valid_idx, 'fold'] = i

    labelencoder= LabelEncoder()
    df['label_group'] = labelencoder.fit_transform(df['label_group'])

    train_df = df[df['fold']!=CFG.TEST_FOLD].reset_index(drop=True)
    train_df = train_df[train_df['fold']!=CFG.VALID_FOLD].reset_index(drop=True)
    valid_df = df[df['fold']==CFG.VALID_FOLD].reset_index(drop=True)
    test_df = df[df['fold']==CFG.TEST_FOLD].reset_index(drop=True)

    train_df['label_group'] = labelencoder.fit_transform(train_df['label_group'])

    return train_df, valid_df, test_df

In [12]:
def precision_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    precision = intersection / len_y_pred
    return precision

def recall_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_true = y_true.apply(lambda x: len(x)).values
    recall = intersection / len_y_true
    return recall

def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [13]:
def get_valid_embeddings(df, model):

    model.eval()

    image_dataset = ShopeeImageDataset(df,transform=get_valid_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.BATCH_SIZE,
        pin_memory=True,
        num_workers = CFG.NUM_WORKERS,
        drop_last=False
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.to(CFG.DEVICE)
            label = label.to(CFG.DEVICE)
            feat,_ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [14]:
def get_valid_neighbors(df, embeddings, KNN = 50, threshold = 0.36):

    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)

    predictions = []
    for k in range(embeddings.shape[0]):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
        
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    df['recall'] = recall_score(df['matches'], df['pred_matches'])
    df['precision'] = precision_score(df['matches'], df['pred_matches'])
    
    del model, distances, indices
    gc.collect()
    return df, predictions

# Training 

In [15]:
def run_training():
    train_df, valid_df, test_df = read_dataset()
    train_dataset = ShopeeDataset(train_df, transform = get_train_transforms())

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = CFG.BATCH_SIZE,
        pin_memory = True,
        num_workers = CFG.NUM_WORKERS,
        shuffle = True,
        drop_last = True
    )

    print(train_df['label_group'].nunique())

    model = ShopeeModel()
    model = replace_activations(model, torch.nn.SiLU, Mish())
    model.to(CFG.DEVICE)

    optimizer = Ranger(model.parameters(), lr = CFG.SCHEDULER_PARAMS['lr_start'])
    #optimizer = torch.optim.Adam(model.parameters(), lr = config.SCHEDULER_PARAMS['lr_start'])
    scheduler = ShopeeScheduler(optimizer,**CFG.SCHEDULER_PARAMS)

    best_valid_f1 = 0.
    for i in range(CFG.EPOCHS):
        avg_loss_train = train_fn(model, train_dataloader, optimizer, scheduler, i)

        valid_embeddings = get_valid_embeddings(valid_df, model)
        valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings)

        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()
        print(f'Valid f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}')

        if valid_f1 > best_valid_f1:
            best_valid_f1 = valid_f1
            print('Valid f1 score improved, model saved')
            torch.save(model.state_dict(),CFG.MODEL_PATH)
        
run_training()

6609
Building Model Backbone for eca_nfnet_l0 model
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.6948816234026509, recall = 0.7134704574806042, precision = 0.8484173363504613
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.7177154513824336, recall = 0.7729562934753996, precision = 0.8157705529797977
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.772187403276379, recall = 0.7466584671046343, precision = 0.9188801503670745
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.777885843546421, recall = 0.7301899104782859, precision = 0.9484926243527251
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.7786400010372356, recall = 0.7193181508970705, precision = 0.9640653112785059
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.7736063256592296, recall = 0.7096930735010765, precision = 0.9684246994356961



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.770657681231268, recall = 0.7035457187860946, precision = 0.9723723875661149



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.7685674280824637, recall = 0.7005650739253617, precision = 0.9726283697300918


In [16]:
def get_test_embeddings(test_df):
    
    model = ShopeeModel()
    model.eval()
    model = replace_activations(model, torch.nn.SiLU, Mish())
    model.load_state_dict(torch.load(CFG.MODEL_PATH))
    model = model.to(CFG.DEVICE)

    image_dataset = ShopeeImageDataset(test_df,transform=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.BATCH_SIZE,
        pin_memory=True,
        num_workers = CFG.NUM_WORKERS,
        drop_last=False
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat,_ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

## Best threshold Search

In [17]:
train_df, valid_df, test_df = read_dataset()

print("Searching best threshold...")
search_space = np.arange(10, 50, 1)

model = ShopeeModel()
model.eval()
model = replace_activations(model, torch.nn.SiLU, Mish())
model.load_state_dict(torch.load(CFG.MODEL_PATH))
model = model.to(CFG.DEVICE)

valid_embeddings = get_valid_embeddings(valid_df, model)

best_f1_valid = 0.
best_threshold = 0.

for i in search_space:
    threshold = i / 100
    valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings, threshold=threshold)

    valid_f1 = valid_df.f1.mean()
    valid_recall = valid_df.recall.mean()
    valid_precision = valid_df.precision.mean()

    print(f"threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

    if (valid_f1 > best_f1_valid):
        best_f1_valid = valid_f1
        best_threshold = threshold

print("Best threshold =", best_threshold)
print("Best f1 score =", best_f1_valid)
BEST_THRESHOLD = best_threshold

Searching best threshold...
Building Model Backbone for eca_nfnet_l0 model



Our image embeddings shape is (6849, 6609)
threshold = 0.1 -> f1 score = 0.6605417999594573, recall = 0.5480996229835974, precision = 0.9981470104334756
threshold = 0.11 -> f1 score = 0.6660601610099165, recall = 0.5552841894670737, precision = 0.9974436183371795
threshold = 0.12 -> f1 score = 0.6711096274056796, recall = 0.5617934363509676, precision = 0.9969801576240475
threshold = 0.13 -> f1 score = 0.6756011665944386, recall = 0.5676809483572353, precision = 0.9963072942047978
threshold = 0.14 -> f1 score = 0.6806218680691374, recall = 0.5743367097513441, precision = 0.9955932834382246
threshold = 0.15 -> f1 score = 0.6862579880780678, recall = 0.5813884608005429, precision = 0.9950332433906682
threshold = 0.16 -> f1 score = 0.6922611920662153, recall = 0.5889756948075289, precision = 0.9946244245850029
threshold = 0.17 -> f1 score = 0.6977711102904753, recall = 0.5961387547689337, precision = 0.9941912713266198
threshold = 0.18 -> f1 score = 0.7017576406884325, recall = 0.6014716

In [18]:
print("Searching best knn...")

search_space = np.arange(10, 50, 2)

best_f1_valid = 0.
best_knn = 0

for knn in search_space:

    valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings, KNN=knn, threshold=BEST_THRESHOLD)

    valid_f1 = valid_df.f1.mean()
    valid_recall = valid_df.recall.mean()
    valid_precision = valid_df.precision.mean()

    print(f"knn = {knn} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

    if (valid_f1 > best_f1_valid):
        best_f1_valid = valid_f1
        best_knn = knn

print("Best knn =", best_knn)
print("Best f1 score =", best_f1_valid)
BEST_KNN = best_knn

Searching best knn...
knn = 10 -> f1 score = 0.78373832304594, recall = 0.7559198838449984, precision = 0.9255959623349028
knn = 12 -> f1 score = 0.7876702682635204, recall = 0.7631287187533448, precision = 0.9235799635383531
knn = 14 -> f1 score = 0.7903063761051121, recall = 0.767520000905081, precision = 0.9226680900903427
knn = 16 -> f1 score = 0.7918399414999109, recall = 0.7700354408053897, precision = 0.9220177518414499
knn = 18 -> f1 score = 0.7928430343120666, recall = 0.7716500380766713, precision = 0.9216830934733095
knn = 20 -> f1 score = 0.7934569660259604, recall = 0.7727130934413423, precision = 0.9213917204210256
knn = 22 -> f1 score = 0.7938791745856082, recall = 0.7734568971456464, precision = 0.9212015008484137
knn = 24 -> f1 score = 0.7943125786603967, recall = 0.7741752589987376, precision = 0.9211107516146617
knn = 26 -> f1 score = 0.7947814802430886, recall = 0.7749185845082397, precision = 0.9211178460435697
knn = 28 -> f1 score = 0.7951271506350569, recall = 0.

In [19]:
test_embeddings = get_valid_embeddings(test_df,model)
test_df, test_predictions = get_valid_neighbors(test_df, test_embeddings, KNN = BEST_KNN, threshold = BEST_THRESHOLD)

test_f1 = test_df.f1.mean()
test_recall = test_df.recall.mean()
test_precision = test_df.precision.mean()
print(f'Test f1 score = {test_f1}, recall = {test_recall}, precision = {test_precision}')


Our image embeddings shape is (6851, 6609)
Test f1 score = 0.7886218847517, recall = 0.77136124813079, precision = 0.9113942995698254
